In [0]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-331-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-cg-toolkit' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common-390' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common-410' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common-415' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common-418' for glob 'nvidia*'
Note, selecting 'nvidi

In [0]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

--2020-05-10 13:29:33--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.0.24
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.0.24|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?OlomHU9B08e_GfkmNvv-8f7cqHAeaO9A5gnSmHYGMmWiGaG2B42J-4qJpv1jGbTlqCYEZlELiVy7LRHXg2o9LqUDfB-oMDv0L_ycWK1oNN6ClsNmMwIcGwA2BWiZefwvqusNa1SCWv5tpLruPNajyuMcZEWsbWM0bfK27IVxm-aWPYP0rdmYasattBNOPGdNu0O-GFR7WIvbc6aDgkw [following]
--2020-05-10 13:29:34--  https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?OlomHU9B08e_GfkmNvv-8f7cqHAeaO9A5gnSmHYGMmWiGaG2B42J-4qJpv1jGbTlqCYEZlELiVy7LRHXg2o9LqUDfB-oMDv0L_ycWK1oNN6ClsNmMwIcGwA2BWiZ

In [0]:
!nvcc --version
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Wed_Apr_11_23:16:29_CDT_2018
Cuda compilation tools, release 9.2, V9.2.88
  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-vx1zjwla
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-vx1zjwla
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=39e179d4d1b99e3869874a93ee4413e6fbf604015f0611458257f14700467b67
  Stored in directory: /tmp/pip-ephem-wheel-cache-jp0w6_nq/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [0]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [13]:
%%cu
#include <bits/stdc++.h>
#include <omp.h>
#include <cuda.h>
#include <stdlib.h>
#include <sys/types.h>
#include <sys/wait.h>
#include <unistd.h>
#include <iostream>
#include <fstream>
#define N 100 

using namespace std;
using namespace std::chrono; 

__device__ float lossgrad(int index, float *A, float *B, float *X){
  float z = 0.0;
  float rho = 0.0;
  float summand = 0.0;
  float grad = 0.0;
  for (int i=0; i<300; i++)
  {  
    z += A[i*100+index]*A[i*100+index];
    summand = 0.0;
    for (int j=0; j<100; j++)
    {
        if(j!= index){
          summand += X[j]*A[i*100+j];
        }
    }
    rho += A[i*100+index]*(B[i] - summand);
  }
  if (rho<-1)
    grad = (rho+1)/z ;
  if (rho> 1)
    grad = (rho-1)/z;
  if(-1<rho && rho<1)
    grad = 0;
  return grad;
}



__global__ void PCDM(float *A, float *B, float *X, int *indices) {
    for(int j=0; j<20; j++){
            X[indices[threadIdx.x+20+j]] = lossgrad(indices[threadIdx.x+20+j], A, B, X);
        }
}

int main(void) {
    ofstream file;
    float *B, *X;
    float *d_B, *d_A , *d_X; 
    int *d_indices, *indices;
    float sizeVec = N * sizeof(float);
    float sizeB = 3*N*sizeof(float);
    float sizeMat = 3*N*N*sizeof(float);
    int sizeInd = N* sizeof(int);
    srand(time(NULL));
    float A[300][100] = {0.0};
    int list_row_cardinality[300] = {0};
    clock_t start = clock();
    cudaMalloc((void **)&d_A, sizeMat);
    cudaMalloc((void **)&d_B, sizeB);
    cudaMalloc((void **)&d_X, sizeVec);
    cudaMalloc((void **)&d_indices, sizeInd);

    X = (float *)malloc(sizeVec);
    for (int i=0; i<100; i++)
      X[i] = (float)0;

    B = (float *)malloc(sizeB); 
    for (int i = 0; i < 300; ++i)
      B[i] = (float) (rand()%100+1);

    file.open("B.txt");
    for(int i=0; i<300; i++)
    {
        file <<B[i]<<" ";
     
    }
    file.close();



    for(int i=0;i<100;i++){
      std::vector< int > v;

      for(int j=0;j<20;j++){
          int random_row = int(rand()%300);

          while((std::count(v.begin(), v.end(), random_row)) != 0 or list_row_cardinality[random_row] >= 35){
              random_row = int(rand()%300);
          }
          
          A[random_row][i] = (float)(rand()%100);
          list_row_cardinality[random_row] ++;
          v.push_back(random_row);

      }
  }  
    file.open("A.txt");
    for(int i=0; i<300; i++){
        for(int j=0; j<100 ;j++)
          {file<< A[i][j]<<" ";}
        file<<'\n';
    }
    file.close();

    indices = (int *)malloc(sizeInd);
    for(int i=0; i<100; i++)
      indices[i] = i;

      
    cudaMemcpy(d_X, X, sizeVec, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, B, sizeB, cudaMemcpyHostToDevice);
    cudaMemcpy(d_A, A, sizeMat, cudaMemcpyHostToDevice);
    file.open("1Threads.txt");
    for(int i=0; i<500; i++)
    {
      random_shuffle(indices,indices+100);
      cudaMemcpy(d_indices, indices, sizeInd, cudaMemcpyHostToDevice);
      PCDM<<<1,1>>>(d_A, d_B, d_X, d_indices);
      cudaMemcpy(X, d_X, sizeVec, cudaMemcpyDeviceToHost);
      for(int i=0; i<100; i++)
        file << X[i]<<" ";
      file<<"\n";
    }
    file.close();
    file.open("8Threads.txt");
    for(int i=0; i<500; i++)
    {
      random_shuffle(indices,indices+100);
      cudaMemcpy(d_indices, indices, sizeInd, cudaMemcpyHostToDevice);
      PCDM<<<1,8>>>(d_A, d_B, d_X, d_indices);
      cudaMemcpy(X, d_X, sizeVec, cudaMemcpyDeviceToHost);
      for(int i=0; i<100; i++)
        file << X[i]<<" ";
      file<<"\n";
    }
    file.close();
    file.open("24Threads.txt");
    for(int i=0; i<500; i++)
    {
      random_shuffle(indices,indices+100);
      cudaMemcpy(d_indices, indices, sizeInd, cudaMemcpyHostToDevice);
      PCDM<<<1,24>>>(d_A, d_B, d_X, d_indices);
      cudaMemcpy(X, d_X, sizeVec, cudaMemcpyDeviceToHost);
      for(int i=0; i<100; i++)
        file << X[i]<<" ";
      file<<"\n";
    }
    file.close();
    cudaMemcpy(X, d_X, sizeVec, cudaMemcpyDeviceToHost);
    free(X); 
    free(B);
    cudaFree(d_X);
    cudaFree(d_A); 
    cudaFree(d_B);
    clock_t end = clock();
    double cpu_time_used = ((double) (end - start)) / CLOCKS_PER_SEC;
    cout<<"Time taken by cuda program : "<<cpu_time_used<<endl;
    return 0;

}


Time taken by cuda program : 28.5777



In [0]:
#How to calc function
#0.5*np.inner(A@X - B, A@X-B) + 1*np.linalg.norm(X,1)

In [0]:
%%cu
#include <bits/stdc++.h>
#include <omp.h> 
#include <stdlib.h>
#include <sys/types.h>
#include <sys/wait.h>
#include <unistd.h>
using namespace std;
using namespace std::chrono; 

vector<int> arr;
const int mod = 1e9+7;

void prn(int * arr, int n){
    for(int i=0 ; i < n ; i++) cout<<arr[i]<<" ";cout<<endl;
}
__global__ void bitonic_sort(int* arr, int j, int k)
{
    int idx = threadIdx.x + blockDim.x * blockIdx.x;
    int i2 = idx ^ j;
    if(i2 > idx)
    {
        if((idx&k)==0)
        {
            if(arr[idx]>arr[i2])
            {
                int temp = arr[idx];
             arr[idx] = arr[i2];
             arr[i2] = temp;
            }
        }
        else{
            if(arr[idx]<arr[i2])
            {
                     int temp = arr[idx];
             arr[idx] = arr[i2];
             arr[i2] = temp;
           
            }
        }
    }
}
int main()
{
    int n= 512*512;
    int next = pow(2, ceil(log(n)/log(2)));
    srand(time(NULL));
    int *cuda_arr;
    int *arr = (int*)malloc(sizeof(int)*next);
    for(int i = 0 ; i < n ; i++)
    {
        arr[i] = rand()%mod;
    }
    for(int i = n ; i < next ; i++)
    {
        arr[i] = 0;
    }
    n = next;
    clock_t start = clock();
    cudaMalloc((void**) &cuda_arr, sizeof(int)*next);
    cudaMemcpy(cuda_arr, arr, sizeof(int)*next, cudaMemcpyHostToDevice);
    
    int BLOCKS = n / 4;
    int THREADS = 4;
    dim3 blocks(BLOCKS,1);    
    dim3 threads(THREADS,1);  
    int j, k;
    for (k = 2; k <= next; k <<= 1) {
        for (j=k>>1; j>0; j=j>>1) {
        bitonic_sort<<<blocks, threads>>>(cuda_arr, j, k);
        }
    }
    cudaMemcpy(arr, cuda_arr, sizeof(int)*next, cudaMemcpyDeviceToHost);
    cudaFree(cuda_arr);
    clock_t end = clock();
 double cpu_time_used = ((double) (end - start)) / CLOCKS_PER_SEC;
    cout<<"Time taken by cuda program : "<<cpu_time_used<<endl;
    prn(arr, n);
}

Time taken by cuda program : 0.138163
4852 11227 12705 13338 15107 15954 18028 18642 19758 21609 22663 23003 23778 26767 28349 30569 30649 32268 32638 35144 39617 39826 43261 50296 51440 53560 55935 56459 60433 64158 65447 66551 66738 67286 69406 70071 70188 79059 85935 96607 97012 98419 98851 102601 105203 106068 108834 109287 109717 109957 111760 112647 117547 118395 120931 130044 134062 137442 138889 140473 146117 147842 148983 151518 155363 155523 163428 163733 173521 175938 179858 181925 182183 182256 188550 197682 199363 200155 201615 202522 203148 208026 210873 212552 220477 222445 222835 223305 223312 225250 227400 227489 228833 230850 230872 232345 232874 236135 239876 241414 243673 255467 271155 276763 279255 279861 282425 282986 284151 289658 293640 294229 300149 301800 304518 304679 306331 309422 312989 313905 313962 319658 320791 321980 327138 338755 339201 339792 342943 348098 349196 351004 351734 364411 365211 370338 377763 381690 382669 383861 389804 392181 396134 40049

In [0]:
# __device__ float lossgrad(int index, float *A, float *B, float *X){
#   float z = 0.0;
#   float rho = 0.0;
#   float summand = 0.0;
#   float grad = 0.0;
#   for (int i=0; i<3000; i++)
#   {  
#     z += A[i*1000+index]*A[i*1000+index];
#     summand = 0.0;
#     for (int j=0; j<1000; j++)
#     {
#         if(j!= index){
#           summand += X[j]*A[i*1000+j];
#         }
#     }
#     rho += A[i*1000+index]*(B[i] - summand);
#   }
#   grad = rho/z ;
#   return grad;
# }


# __global__ void PCDM(float *A, float *B, float *X, int *indices) {
#     for (int i=0; i<1000; i++){
#         printf("thread:%d,i:%d\n",threadIdx.x,i);
#         for(int j=0; j<20; j++){
#             X[indices[blockIdx.x+j]] = lossgrad(indices[blockIdx.x+j], A, B, X);
#         }
#     }
# }
 